# Empresa transportadora de óleos
## Alunos

| Nome                        | Matrícula |
|-----------------------------|-----------|
| Luiz Filipe Bartelega Penha | 202111082 |
| Vitor Pires Zini            | 202110169 |

In [ ]:
!pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.4/14.4 MB 24.1 MB/s eta 0:00:00


In [ ]:
from gurobipy import Model, GRB

modelo = Model("Transporte óleo")

Restricted license - for non-production use only - expires 2026-11-23


In [ ]:
AN_var = modelo.addVar(name="a->norte", vtype=GRB.CONTINUOUS, lb=0)
AS_var = modelo.addVar(name="a->sul", vtype=GRB.CONTINUOUS, lb=0)

SN_var = modelo.addVar(name="sul->norte", vtype=GRB.CONTINUOUS, lb=0)

NB_var = modelo.addVar(name="norte->b", vtype=GRB.CONTINUOUS, lb=0)
SB_var = modelo.addVar(name="sul->b", vtype=GRB.CONTINUOUS, lb=0)

In [ ]:
modelo.addConstr(AN_var <= 300, name="fluxo an")
modelo.addConstr(AS_var <= 500, name="fluxo as")
modelo.addConstr(SN_var <= 300, name="fluxo sn")
modelo.addConstr(NB_var <= 400, name="fluxo nb")
modelo.addConstr(SB_var <= 300, name="fluxo sb")

modelo.addConstr(AN_var + SN_var == NB_var, name="conservação fluxo norte")
modelo.addConstr(AS_var == SN_var + SB_var, name="conservação fluxo sul")

<gurobi.Constr *Awaiting Model Update*>

In [ ]:
modelo.setObjective(NB_var + SB_var, GRB.MAXIMIZE)
modelo.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 7 rows, 5 columns and 11 nonzeros
Model fingerprint: 0xdfb8879e
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+02, 5e+02]
Presolve removed 5 rows and 1 columns
Presolve time: 0.01s
Presolved: 2 rows, 4 columns, 5 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.0000000e+02   1.250000e+01   0.000000e+00      0s
       1    7.0000000e+02   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.02 seconds (0.00 work units)
Optimal objective  7.000000000e+02


In [ ]:
if(modelo.status == GRB.OPTIMAL):
  print(f"an: {AN_var.X}")
  print(f"as: {AS_var.X}")
  print(f"sn: {SN_var.X}")
  print(f"nb: {NB_var.X}")
  print(f"sb: {SB_var.X}")
  print(f"Valor ótimo: {modelo.objVal}")


an: 300.0
as: 400.0
sn: 100.0
nb: 400.0
sb: 300.0
Valor ótimo: 700.0


In [ ]:
print("\nIntervalos de Custos (Vetor de Custos):")
for variavel in modelo.getVars():
  print(f"Variável {variavel.VarName}:")
  print(f"  lower bound do custo (SAObjLow): {variavel.SAObjLow}")
  print(f"  upper bound do custo (SAObjUp): {variavel.SAObjUp}")

print("\nIntervalos de Recursos (Vetor de Recursos):")
for restricao in modelo.getConstrs():
  print(f"Restrição {restricao.ConstrName}:")
  print(f"  lower bound do recurso (SARHSLow): {restricao.SARHSLow}")
  print(f"  upper bound do recurso (SARHSUp): {restricao.SARHSUp}")
  print(f"  Variavel dual '{restricao.ConstrName}': Dual = {restricao.Pi}")



Intervalos de Custos (Vetor de Custos):
Variável a->norte:
  lower bound do custo (SAObjLow): -0.0
  upper bound do custo (SAObjUp): inf
Variável a->sul:
  lower bound do custo (SAObjLow): -1.0
  upper bound do custo (SAObjUp): 0.0
Variável sul->norte:
  lower bound do custo (SAObjLow): -1.0
  upper bound do custo (SAObjUp): 0.0
Variável norte->b:
  lower bound do custo (SAObjLow): -0.0
  upper bound do custo (SAObjUp): inf
Variável sul->b:
  lower bound do custo (SAObjLow): -0.0
  upper bound do custo (SAObjUp): inf

Intervalos de Recursos (Vetor de Recursos):
Restrição fluxo an:
  lower bound do recurso (SARHSLow): 200.0
  upper bound do recurso (SARHSUp): 400.0
  Variavel dual 'fluxo an': Dual = 0.0
Restrição fluxo as:
  lower bound do recurso (SARHSLow): 400.0
  upper bound do recurso (SARHSUp): inf
  Variavel dual 'fluxo as': Dual = 0.0
Restrição fluxo sn:
  lower bound do recurso (SARHSLow): 100.0
  upper bound do recurso (SARHSUp): inf
  Variavel dual 'fluxo sn': Dual = 0.0
Res

# Análise de sensibilidade
Primeiramente, avaliando os intervalos de custos, observa-se que as variáveis "a -> norte", "norte -> b" e "sul -> b" possuem grande flexibilidade, pois seus custos podem aumentar indefinidamente sem alterar a solução ótima, mas não podem ser reduzidos abaixo de -0.0. Por outro lado, as variáveis "a -> sul" e "sul -> norte" possuem intervalos mais restritos, variando entre -1.0 e 0.0. Isso indica que essas duas últimas variáveis são mais sensíveis a mudanças nos custos, podendo impactar a solução ótima caso os valores saiam desses intervalos.

No que diz respeito aos intervalos de recursos, os resultados mostram que a maioria das restrições apresenta valores duals iguais a 0.0, indicando que essas restrições não estão sendo críticas na solução ótima. Por exemplo, a restrição "fluxo an" permite que os recursos variem entre 200.0 e 400.0 sem impactar a solução, enquanto "fluxo nb" aceita variações entre 300.0 e 500.0. Algumas restrições, como "fluxo as", "fluxo sn" e "fluxo sb", não possuem limites superiores definidos, o que demonstra flexibilidade para aumentos nos recursos. Já as restrições de conservação de fluxo ("norte" e "sul") apresentam intervalos mais amplos, permitindo variações significativas sem alterar a base ótima.